In [2]:
import tensorflow

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.utils import np_utils
import tensorflow as tf
# import cv2
import numpy as np
# import pandas as pd
import os

In [4]:
import pandas as pd

In [5]:
from sklearn import preprocessing

In [6]:
import cv2

In [7]:
import math

In [8]:
csv_file = "data_collection.csv"
columns = ['Serial_no','Timestamp', 'x_actual', 'y_actual', 'direction', 'x_step', 'y_step']
data_df = pd.read_csv(csv_file, header=None, names = columns)

serial_numbers = data_df['Serial_no'].astype(str).apply(lambda x: int(x.split(',')[0]))

image_folder = "output_images_new"#"cropped_noedges_new" #"output_images_new" #"images_resized_500"
imagess = []
parsed_custom_info = []

le = preprocessing.LabelEncoder()
data_df['direction'] = le.fit_transform(data_df['direction'])

# for index, info in enumerate(data_df[0]):



# serial_number = data_df['Serial_no.']
serial_numbers = set(data_df['Serial_no'])

date_time = data_df['Timestamp']
x_coordinate = data_df['x_actual']
y_coordinate = data_df['y_actual']
direction = data_df['direction']
x_step = data_df['x_step']
y_step = data_df['y_step']

data_df = data_df[3000:]

image_files = os.listdir(image_folder)
imagess = []

for row in data_df.itertuples(index=False, name='data_df'):
    serial_number = row.Serial_no
    date_time = row.Timestamp
    x_coordinate = row.x_actual
    y_coordinate = row.y_actual
    direction = row.direction
    x_step = row.x_step
    y_step = row.y_step

    # if ~(pd.isna(x_coordinate) and pd.isna(y_coordinate)):
    # if ~(pd.isnull(x_coordinate) and pd.isnull(y_coordinate)):
    # if pd.notnull(x_coordinate) and pd.notnull(y_coordinate):
    # if not math.isnan(x_coordinate) and not math.isnan(y_coordinate):
    # if not x_coordinate.empty and not y_coordinate.empty:
    # if not(direction == 0 and x_coordinate > 4.49) or not(direction == 1 and y_coordinate < -18.81):

    #############################
    if pd.notnull(x_coordinate) and pd.notnull(y_coordinate) and \
        not((direction == 0 and (x_coordinate > 4.49 or x_coordinate < -12.5)) \
               or (direction == 1 and (x_coordinate < -18.81 or x_coordinate > -3.075))):
            matching_files = [filename for filename in image_files if filename.startswith(str(serial_number))]
            if matching_files:
                image_name = matching_files[0]  # Assuming there is only one matching file
                image_path = os.path.join(image_folder, image_name)
                image = cv2.imread(image_path)
                # if image is not None: 
                imagess.append(image)
                parsed_custom_info.append([x_coordinate, y_coordinate, direction])

In [ ]:
# max_value = data_df['column_name'].max()

# # Find the minimum value
# min_value = data_df['column_name'].min()

# print("Maximum value:", max_value)
# print("Minimum value:", min_value)

In [16]:
print(data_df.shape)

(3930, 7)


In [47]:
# grouped = data_df.groupby(['direction'])
# df1 = grouped.get_group((0))
# df2 = grouped.get_group((1))

In [48]:
# print(df1.shape)
# print(df2.shape)

(1960, 7)
(1970, 7)


In [46]:
# print(data_df['direction'].isnull().sum())

0


In [18]:
!nvidia-smi

Wed Jul 19 13:22:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX 6000...  On   | 00000000:41:00.0 Off |                  Off |
| 30%   38C    P8    21W / 300W |      1MiB / 49140MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX 6000...  On   | 00000000:61:00.0 Off |                  Off |
| 30%   

In [19]:
data = list(zip(imagess, parsed_custom_info))


In [20]:
# from keras.backend.tensorflow_backend import set_session
from keras.backend import set_session

In [21]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [22]:
config = tf.ConfigProto()
# config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
set_session(sess)


2023-07-19 13:22:32.198062: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-19 13:22:32.198264: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-19 13:22:32.213465: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-19 13:22:32.213711: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-19 13:22:32.213870: I tensorflow/compiler/xla/stream_executo

In [13]:
# print(len(data))

In [23]:
# Shuffle and split the dataset
np.random.shuffle(data)
train_ratio = 0.9
train_size = int(len(data) * train_ratio)

train_data = data[:train_size]
test_data = data[train_size:]

X_train = [item[0] for item in train_data]
Y_train = [item[1] for item in train_data]

X_test = [item[0] for item in test_data]
Y_test = [item[1] for item in test_data]

In [25]:
X_test1 = np.array(X_test)
X_train1 = np.array(X_train)

In [28]:
X_test1 = X_test1.astype('float32')
X_train1 = X_train1.astype('float32')

In [30]:
X_test1 /= 255

In [32]:
X_train1 /= 255

In [33]:
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

In [23]:
# check_nan = data_df['x_actual'].isnull().values.any()
# print(check_nan)

In [24]:
# if np.isnan(parsed_custom_info).any():
    # print("The array contains NaN values.")

In [25]:
# print(type(parsed_custom_info))

In [26]:
# print("Shape X_train: ", np.shape(X_train1))
# print("Shape Y_train: ", np.shape(Y_train))

# print("Shape X_test: ", np.shape(X_test1))
# print("Shape Y_test: ", np.shape(Y_test))



In [35]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [36]:
from keras.layers import MaxPooling2D

In [29]:
# Y_train = np.nan_to_num(Y_train)


In [30]:
# import tensorflow as tf
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     try:
#         tf.config.experimental.set_memory_growth(gpus[0], True)
#     except RuntimeError as e:
#         print(e)


In [37]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(1080, 1920, 3), kernel_regularizer=tf.keras.regularizers.l2(0.001))) #(1080, 1920, 3) #(1080, 1350, 3)
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(Flatten())
model.add(Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(Dense(3))  # Output layer with 3 units for regression

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, clipnorm=1.0)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Train the model
history_left = model.fit(X_train1, Y_train, epochs=150, batch_size=32)

Train on 862 samples
Epoch 1/150


2023-07-19 13:23:11.091126: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2023-07-19 13:23:11.220320: W tensorflow/c/c_api.cc:291] Operation '{name:'training/Adam/conv2d/bias/v/Assign' id:367 op device:{requested: '', assigned: ''} def:{{{node training/Adam/conv2d/bias/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training/Adam/conv2d/bias/v, training/Adam/conv2d/bias/v/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-07-19 13:23:13.945526: W tensorflow/core/kernels/gpu_utils.cc:50] Failed to allocate memory for convolution redzone checking; skipping this check. This is benign and only means that we won't check cudnn for out-of-bounds reads and writes. This message will only be print

862/862 [==============================] - 47s 54ms/sample - loss: 9728.8331
Epoch 2/150
862/862 [==============================] - 29s 33ms/sample - loss: 27.5688
Epoch 3/150
862/862 [==============================] - 29s 33ms/sample - loss: 24.0417
Epoch 4/150
862/862 [==============================] - 29s 33ms/sample - loss: 20.9468
Epoch 5/150
862/862 [==============================] - 28s 33ms/sample - loss: 18.5101
Epoch 6/150
862/862 [==============================] - 29s 33ms/sample - loss: 18.8044
Epoch 7/150
862/862 [==============================] - 29s 33ms/sample - loss: 17.7995
Epoch 8/150
862/862 [==============================] - 28s 33ms/sample - loss: 20.1898
Epoch 9/150
862/862 [==============================] - 29s 33ms/sample - loss: 16.9516
Epoch 10/150
862/862 [==============================] - 28s 33ms/sample - loss: 18.6249
Epoch 11/150
862/862 [==============================] - 28s 33ms/sample - loss: 18.8403
Epoch 12/150
862/862 [=============================

In [32]:
# model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(1080, 1920, 3), kernel_regularizer=tf.keras.regularizers.l2(0.001)))


In [33]:
# model.add(MaxPooling2D((2, 2)))


In [34]:
# model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))


In [35]:
# model.add(MaxPooling2D((2, 2)))


In [36]:
# model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))


In [37]:
# model.add(Flatten())


In [38]:
# model.add(Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))


In [39]:
# model.add(Dense(3))  # Output layer with 3 units for regression


In [40]:
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, clipnorm=1.0)


In [41]:
# model.compile(optimizer=optimizer, loss='mean_squared_error')


In [24]:
model.save("removednoncontactdatacompletely_model_150ep.h5")


In [2]:
!nvidia-smi

Wed Jul 19 17:37:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX 6000...  On   | 00000000:41:00.0 Off |                  Off |
| 30%   38C    P8    21W / 300W |      1MiB / 49140MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX 6000...  On   | 00000000:61:00.0 Off |                  Off |
| 30%   

In [26]:
import matplotlib.pyplot as plt

In [ ]:
# Obtain the loss values from the training history
loss_values = history.history['loss']

# Plot the loss graph
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, 'b', label='Training Loss')
plt.title('Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
# plt.xlim(0, len(loss_values) + 1)
# plt.ylim(0, max(loss_values) + 100)
plt.legend()
plt.show()

In [ ]:
# Obtain the loss values from the training history
loss_values = history.history['loss']

# Plot the loss graph
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, 'b', label='Training Loss')
plt.title('Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
# plt.xlim(0, len(loss_values) + 1)
# plt.ylim(0, max(loss_values) + 100)
plt.ylim(0, 30)

plt.legend()
plt.show()

In [29]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [30]:
#Evaluation

Y_pred = model.predict(X_test1)

mse = mean_squared_error(Y_test, Y_pred)

rmse = np.sqrt(mse)

mae = mean_absolute_error(Y_test, Y_pred)

r2 = r2_score(Y_test, Y_pred)

print('Mean Squared Error (MSE):', mse)
print('Root Mean Squared Error (RMSE):', rmse)
print('Mean Absolute Error (MAE):', mae)
print('R-squared (R2):', r2)

/usr/lib/python3/dist-packages/keras/engine/training_v1.py:2357: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2023-07-19 11:19:51.626281: W tensorflow/c/c_api.cc:291] Operation '{name:'dense_3/BiasAdd' id:271 op device:{requested: '', assigned: ''} def:{{{node dense_3/BiasAdd}} = BiasAdd[T=DT_FLOAT, _has_manual_control_dependencies=true, data_format="NHWC"](dense_3/MatMul, dense_3/BiasAdd/ReadVariableOp)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Mean Squared Error (MSE): 6.3553212282477745
Root Mean Squared Error (RMSE): 2.52097624507804
Mean Absolute Error (MAE): 1.296749566704765
R-squared (R2): 0.6020065608068376


In [ ]:
sample_index = 6  # Replace with the index of the desired sample

sample = np.expand_dims(X_test1[sample_index], axis=0) 

y_pred = model.predict(sample)

print('Prediction:', y_pred)

In [ ]:
print(Y_test[6])

In [ ]:
sample_index = 15  # Replace with the index of the desired sample

sample = np.expand_dims(X_test1[sample_index], axis=0) 

y_pred = model.predict(sample)

print('Prediction:', y_pred)

In [ ]:
print(Y_test[15])

In [ ]:
sample_index = 32  # Replace with the index of the desired sample

sample = np.expand_dims(X_test1[sample_index], axis=0) 

y_pred = model.predict(sample)

print('Prediction:', y_pred)

In [ ]:
print(Y_test[32])

In [31]:
sess.close()
tf.reset_default_graph()


In [36]:
!nvidia-smi

Wed Jul 19 04:48:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX 6000...  On   | 00000000:41:00.0 Off |                  Off |
| 30%   37C    P8    21W / 300W |  47843MiB / 49140MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX 6000...  On   | 00000000:61:00.0 Off |                  Off |
| 30%   